#### 여기서는 Tutorial에서 배운 개념을 이용하여 간단하게 ReplayBuffer를 분산 환경에서 활용해보겠습니다. <br> 아래와 같은 작업을 수행합니다. <br>  
    1. 여럿의 agent(혹은 actor)가 공유 Replay Buffer에 경험데이터를 넣는다. 
    2. Learner는 batch만큼 그 공유 ReplayBuffer에서 load한 후 원하는 작업을 수행한다.

#### 질문 <br>

    1. Class의 method는 공유가 잘 되는데, class 안에 있는 __init__ 에서 선언된 variable은 불러올 수가 없었다. 어떻게 해야하는 걸까?
       : 현재로써는 class 안에 변수를 전달하는 method를 따로 만들어서 ray.get으로 접근하는 방법을 쓰고있음..

In [1]:
import ray 
import time 
import numpy as np 

In [2]:
ray.init() 

2021-01-28 18:13:07,511	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '192.168.0.61',
 'raylet_ip_address': '192.168.0.61',
 'redis_address': '192.168.0.61:44383',
 'object_store_address': '/tmp/ray/session_2021-01-28_18-13-06_812268_99563/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-28_18-13-06_812268_99563/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-01-28_18-13-06_812268_99563',
 'metrics_export_port': 56253,
 'node_id': 'bee19ef478404e9e4b5159c0980835a49d4e561d'}

In [3]:
# 간단한 env를 정의하겠습니다. environment의 일반적인 메소드만 넣고 어떤 의미가 있는 행동이나 상태를 정의한 것은 아닙니다.
class Env:        
    def reset(self):
        return np.ones((2,2))

    def step(self, action):
        # state, reward, done 모두 random하게 지정. state의 크기는 2x2 차원을 가지는 2차원 matrix.
        state = action*np.random.randn(2, 2)
        reward = np.sum(state)

        # done은 numpy의 random.randn 이 0.06 보다 작을 때만 1을 주었습니다. 더 자주 done이 발생하도록 하고 싶다면, 0.06을 더 키우면 됩니다.
        done = 1 if abs(np.random.randn())<0.06 else 0
        return state, reward, done

In [4]:
# Buffer를 정의합니다.
@ray.remote
class Buffer:
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.state_buffer = np.zeros((buffer_size, 2 ,2))
        self.action_buffer = np.zeros(buffer_size)
        self.reward_buffer = np.zeros(buffer_size)
        self.next_state_buffer = np.zeros((buffer_size, 2 ,2))
        self.done_buffer = np.zeros(buffer_size)
        self.act_idx_buffer = np.zeros(buffer_size)
        
        self.store_idx = 0
        self.current_size = 0
        self.total_store_count = 0

    def store(self, state, action, next_state, reward, done, actor_idx):
        self.state_buffer[self.store_idx] = state
        self.action_buffer[self.store_idx] = action
        self.reward_buffer[self.store_idx] = reward
        self.next_state_buffer[self.store_idx] = next_state
        self.done_buffer[self.store_idx] = done
        self.act_idx_buffer[self.store_idx] = actor_idx
        
        self.store_idx = (self.store_idx + 1) % self.buffer_size
        self.current_size = min(self.current_size+1, self.buffer_size)
        self.total_store_count += 1
    
    def batch_load(self, batch_size): 
        indices = np.random.randint(self.current_size, size=batch_size)  
        return dict( 
                states=self.state_buffer[indices], 
                actions=self.action_buffer[indices],
                rewards=self.reward_buffer[indices],
                next_states=self.next_state_buffer[indices], 
                dones=self.done_buffer[indices],
                actindices=self.act_idx_buffer[indices])  
    
    def return_current_size(self):
        return self.total_store_count

In [5]:
# actor의 역할은 각각 env에서 경험한 것을 buffer에 넘겨주는 역할을 합니다.
@ray.remote
class Actor:
    def __init__(self, memory, actor_idx):
        self.env = Env() 
        self.memory = memory # ray를 통해 공유하는 learner class입니다.
        self.actor_idx = actor_idx # 어떤 actor에서 온 데이터인지 보기 위한 변수입니다.

    def explore(self):
        state = self.env.reset()
        # actor는 멈추지 않아도 되기 때문에, 다음과 같이 무한 loop로 exploration하도록 설정
        while 1:
            action = np.random.randint(3) 
            next_state, reward, done = self.env.step(action) 
            
            # 공유 메모리에 데이터를 저장합니다
            self.memory.store.remote(state, action, next_state, reward, done, self.actor_idx) 
            
            state = next_state
            time.sleep(0.005)
            if done:
                state = self.env.reset() 

In [6]:
# 공유 Buffer를 통해 학습을 진행하는 Learner를 정의합니다.
class Learner:
    def __init__(self, memory, buffer_size, batch_size):
        self.memory = memory
        self.batch_size = batch_size

    def update_network(self):
        # 저장된 buffer에서 데이터를 로딩합니다.
        batch = ray.get(self.memory.batch_load.remote(self.batch_size))
        print("batch is loaded.")
        ''' update를 하는 부분 '''
        
        loss = np.random.randn()
        buffer_store_count = ray.get(self.memory.return_current_size.remote())
        return loss, batch['states'].shape, batch['actindices'], buffer_store_count # 결과를 확인하기 위해서, loss 이외에 몇 가지를 추가

In [7]:
buffer_size = 5000 # Replay Buffer 사이즈
batch_size = 16    # Replay Buffer에서 가지고 올 샘플 개수

memory = Buffer.remote(batch_size)
learner = Learner(memory, buffer_size, batch_size) 

In [8]:
num_actors = 5 # actor의 개수

# num_actors 개수만큼 선언하고, explore 실행. actor라는 변수가 계속 중복이 되지만 실행은 잘 된다.
for idx in range(num_actors):
    actor = Actor.remote(memory, idx)
    actor.explore.remote()
time.sleep(1) # 잠시 actor가 어느정도 쌓을 때까지 대기

In [ ]:
n_updates = 100 # learner가 update_network 메소드를 실행하는 횟수

for update_idx in range(n_updates): 
    loss, batch_stat_shape, act_indices, buf_size = learner.update_network()
    print(f'Number of updates: {update_idx}')
    print(f'Loss: {loss}')
    print(f'State shape in Batch: {batch_stat_shape}')
    print(f'Actor index: {act_indices}')
    print(f'Buffer store index: {buf_size}\n')
    time.sleep(0.1)

batch is loaded.
Number of updates: 0
Loss: 1.4752570594307386
State shape in Batch: (16, 2, 2)
Actor index: [3. 3. 1. 3. 4. 0. 3. 3. 0. 3. 3. 4. 1. 0. 3. 3.]
Buffer store index: 773

batch is loaded.
Number of updates: 1
Loss: 0.41636878786804854
State shape in Batch: (16, 2, 2)
Actor index: [0. 2. 2. 3. 2. 3. 2. 4. 4. 0. 0. 2. 2. 3. 0. 3.]
Buffer store index: 1564

batch is loaded.
Number of updates: 2
Loss: 0.21922156767472734
State shape in Batch: (16, 2, 2)
Actor index: [3. 2. 2. 3. 0. 0. 1. 1. 4. 0. 4. 3. 3. 3. 3. 2.]
Buffer store index: 2412

batch is loaded.
Number of updates: 3
Loss: -0.05244928260209676
State shape in Batch: (16, 2, 2)
Actor index: [4. 4. 3. 0. 4. 4. 4. 4. 1. 0. 1. 3. 4. 0. 3. 0.]
Buffer store index: 2913

batch is loaded.
Number of updates: 4
Loss: 1.6871242788143255
State shape in Batch: (16, 2, 2)
Actor index: [4. 0. 0. 4. 0. 0. 4. 3. 2. 4. 1. 1. 0. 2. 4. 3.]
Buffer store index: 3229

batch is loaded.
Number of updates: 5
Loss: -1.9292772107110407
State sh


- Loss: random한 실수값 <br>
- State shape: (batch, state[0], state[1])의 자원을 가지는 출력 <br>
- Actor index: batch 안의 각 sample이 어느 actor에게 나온 것인지 출력 <br>
- Buffer store index: Buffer에 저장되는 현재 store index(각 update 사이에 얼마나 저장되었는지)를 출력  <br><br>

#### 대략 아래와 같은 결과가 나오면 의도대로 나온 것입니다. 

    Number of updates: 9
    Loss: -1.7283143861676746
    State shape in Batch: (16, 2, 2)
    Actor index: [ 4. 12.  1.  3.  4.  4.  1. 14.  2. 15. 11.  0.  1. 15. 15.  9.]
    Buffer store index: 1863

    Number of updates: 10
    Loss: -1.3466382853532786
    State shape in Batch: (16, 2, 2)
    Actor index: [ 9.  8. 13. 15. 14.  9.  0.  4.  2.  8. 13.  7.  2.  2.  0. 11.]
    Buffer store index: 2023

    Number of updates: 11
    Loss: -0.8023523911669711
    State shape in Batch: (16, 2, 2)
    Actor index: [ 3.  9.  9.  7. 12.  3. 12.  6. 12.  5. 10.  7.  0. 11.  3.  6.]
    Buffer store index: 2181